In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
import torch.nn as nn
import re
import warnings

In [ ]:
!pip3 install rouge-score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def find_max_similarity(answer):
  segments = [s.strip() for s in re.split('[,.!?]', answer) if len(s.strip().split()) >= 2]
  max_score, s1, s2 = 0, "", ""
  if len(segments) > 0:
    for i in range(len(segments)):
      for j in range(i + 1, len(segments)):
        score = scorer.score(segments[i], segments[j])['rougeL'].fmeasure
        if score > max_score: max_score, s1, s2 = score, segments[i], segments[j]
  return (max_score, s1, s2)

In [ ]:
from google.colab import drive
drive.mount ("/content/drive")
import os
os.chdir("/content/drive/MyDrive/TEST")
os.environ['WANDB_MODE'] = 'disabled'
model_name="./MODEL/qa_model"
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_QA = pd.read_csv("QA_pairs.csv")
df_QA2 = pd.read_csv("QA_pairs_simple.csv")

In [ ]:
model_name = "./MODEL/qa_model"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

In [ ]:
def predict(question, answer, model, tokenizer):
    text = f"Question: {question} Answer: {answer}"
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model = model.cuda()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits.float(), dim=-1)
    label, confidence = int(predictions.argmax().item()), round(predictions.max().item(),3)
    return label, confidence

In [ ]:
def clean_text(text):
   text = text.replace('\n', ' ')
   text = text.replace('<userStyle>Normal</userStyle>', '')
   text = ' '.join(text.split())
   return text
combined_df = pd.DataFrame({
   'question': df_QA['Question'],
   'answer_LLAMA': df_QA['LLAMA_Finetuned'].apply(clean_text),
   'summary_bart': df_QA2['answers_simple1_bart_large_cnn'].apply(clean_text),
   'summary_llama': df_QA2['answers_simple2_llama'].apply(clean_text)
})
def get_final_label(question, answer):
   label, _ = predict(question, answer, model, tokenizer)
   max_score, _, _ = find_max_similarity(answer)
   if label == 1 and max_score < 0.65:
       return 1
   return 0
combined_df['label_LLAMA'] = [
   get_final_label(q, a)
   for q, a in zip(combined_df['question'], combined_df['answer_LLAMA'])
]
combined_df['label_bart'] = [
   get_final_label(q, a)
   for q, a in zip(combined_df['question'], combined_df['summary_bart'])
]
combined_df['label_llama'] = [
   get_final_label(q, a)
   for q, a in zip(combined_df['question'], combined_df['summary_llama'])
]

In [ ]:
len(combined_df[combined_df['label_LLAMA'] == 1]), len(combined_df[combined_df['label_bart'] == 1]), len(combined_df[combined_df['label_llama'] == 1])

(11, 99, 87)

In [ ]:
combined_df.to_csv("QA_pairs_scoring_byfittedmodel2.csv",index=False)
combined_df

,question,answer_LLAMA,summary_bart,summary_llama,label_LLAMA,label_bart,label_llama
0,"Why is the ""Black Death"" mentioned so often in...",The Black Death is a very famous event in Euro...,"The ""Black Death"" originated in the Middle Eas...","The ""Black Death"" is a historical phenomenon t...",0,1,1
1,How historically accurate is the show The Last...,I've watched the first season of the show and ...,The Last Kingdom is a historical drama series ...,"The Last Kingdom, a historical drama series, i...",0,1,1
2,If there was a lack of food supply in the worl...,There's a lot of food out there. You just need...,Vitamins are crucial for various bodily functi...,Humans need a balanced diet that includes vari...,0,1,1
3,"If our DNA varies from person to person, how d...","Humans have 23 pairs of chromosomes, and each ...",The first step is to sequence the DNA of a hum...,Researchers compare human DNA to that of orang...,0,1,1
4,Why would a company spend millions on a long S...,"If you want to get a lot of attention, you wan...",Companies often spend millions on Super Bowl c...,Companies often spend millions on Super Bowl c...,0,1,1
...,...,...,...,...,...,...,...
95,The difference between a learning disability a...,"A learning disability is a learning disorder, ...",A learning disability is a neurological or psy...,Learning disabilities and intellectual disabil...,0,1,1
96,Why do I get headaches from 3D movies?,The reason why you get headaches from 3D movie...,There are several reasons why you might experi...,Watching 3D movies can cause headaches due to ...,0,1,0
97,Why do people have different skin colours?,Humans are a lot like plants. We have a lot of...,The skin color of a person is determined by th...,The skin color of a person is determined by th...,0,1,1
98,"How did girlscout cookies get to be so good, a...",The Girl Scouts have a strict recipe that they...,Girl Scout cookies have been a staple in the o...,The Girl Scout cookies have been a beloved tre...,0,1,1
